# Test algorithms

## Policy Gradient - Continuous Action

In [1]:
%run ./repos/mbzuai-rl-research/cont-pg-simple.py -b 128

/home/nicolas.avila/.conda/envs/rl_energy/lib/python3.8/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: madog. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nicolas.avila/.netrc
/home/nicolas.avila/.conda/envs/rl_energy/lib/python3.8/site-packages/wandb/sdk/lib/ipython.py:47: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML  # type: ignore


## Advantage Actor Critic (A2C) Simple - Continuous Action

In [2]:
%run ./repos/mbzuai-rl-research/cont-a2c-simple.py

/home/nicolas.avila/.conda/envs/rl_energy/lib/python3.8/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: madog. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nicolas.avila/.netrc
/home/nicolas.avila/.conda/envs/rl_energy/lib/python3.8/site-packages/wandb/sdk/lib/ipython.py:47: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML  # type: ignore


actor_loss,▁▁▂▂▂▁▂▂▂▃▄▄▅▆▆▆▇██▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
critic_loss,█▇▇▇▆▆▅▄▃▂▂▁▁▁▁▁▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
rollout_avg_reward,▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▄▄▆▇█████████████████████
actor_loss,-0.17345
critic_loss,0.88159
rollout_avg_reward,-1.64285


## Advantage Actor Critic (A2C) Causality Simple - Continuous Action

In [2]:
%run ./repos/mbzuai-rl-research/cont-a2c-caus-simple.py -dl